In [1]:
import pymongo
import pandas as pd
import re
import time
import datetime
from datetime import datetime, timedelta
import nltk

In [37]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
threads = db.threads
companies = db.companylist

In [3]:
df = pd.DataFrame.from_records(threads.find({'Label' :{'$ne': 0}}))

In [32]:
my_sent = df['Body'][2]

In [33]:
def get_continuous_chunks(text):
    chunked = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == nltk.tree.Tree and i.label() == 'ORGANIZATION':
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
    return continuous_chunk

In [34]:
get_continuous_chunks(my_sent)

['Channel Support',
 'SPY',
 'Vaccine News Can',
 'OPEX',
 'SPY Regardless',
 'MMs',
 'IMO',
 'TLDR']

In [35]:
my_sent

'Last week I shared my S P500 forecast and broke down the technicals that prompted my bullish outlook for this week. See my previous reddit post here https //www.reddit.com/r/wallstreetbets/comments/j162ms/sp500 technical analysis chart breakdown stay long/ https //www.reddit.com/r/wallstreetbets/comments/j162ms/sp500 technical analysis chart breakdown stay long/ We ended up blowing through all of the long targets by the end of Monday cash session. Before https / /www.tradingview.com /chart /SPX500USD /lm8lMR1i S P500 ES 4H Bullish Engulfing Candle at Channel Support / https //preview.redd.it/1peue59jkfq51.png width 3040 format png auto webp s fca01dba94c7bed1c2864b5b22b1d18c91f8ce1f After https //preview.redd.it/7f53pyeokfq51.png width 3044 format png auto webp s 02cb0b0b0b2e4aba5145c306244dd8acf5527688 I remain extremely bullish in my most recent forecast below with an extended target on ES of $3500 which is $350 for SPY. I think we will reach the initial 3420 target by the end of th

In [38]:
companydf = pd.DataFrame.from_records(companies.find())

In [40]:
companydf['Name']

0                           10x Genomics, Inc.
1       1347 Property Insurance Holdings, Inc.
2       1347 Property Insurance Holdings, Inc.
3                     180 Degree Capital Corp.
4                      1-800-FLOWERS.COM, Inc.
                         ...                  
5716                               Zumiez Inc.
5717                            Zymeworks Inc.
5718             Zynerba Pharmaceuticals, Inc.
5719                               Zynex, Inc.
5720                                Zynga Inc.
Name: Name, Length: 5721, dtype: object